# Muestreo Centros Digitales

### Cambios en esta versión
* Validar retricción de no más de 20 (parametro) satelitales por hora
### Próximos pasos
* Exploración datos por satelitales
* Sacar parametro porcentaje de muestra
 * Validar que cumplan 303 pruebas de 1515 aps

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import date
import datetime
import random
import math

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Parámetro para definir el medio de carga de datos

In [4]:
formas_de_ingreso=['db','archivos']
modoIngresoInformacion=formas_de_ingreso[0] #cambia a 1 si se usan archivos

In [5]:
con_graficas=False
num_horas=15
pc_muestra=0.3
n_beams_sat=18

## Lectura Bases de datos

In [6]:
if modoIngresoInformacion== 'db':
    ## Llamado funcion para consulta y parametros
    """
    from get_data_from_db import *    
    registros, aps_operativos, num_aps, num_muestras_hora=GetDataFromDB()
    str_list=["ID","BANDA","SUBIDA_LIM","BAJADA_LIM","FECHA","HORA","CONDICION","VELOCIDAD_BAJADA","VELOCIDAD_SUBIDA","CUMPLE_BANDA","CUMPLE_SUBIDA","CUMPLE_BAJADA",'BASE','FASE']
    Data=pd.DataFrame.from_records(registros, columns=str_list)
    """
    #Carga Archivos Prueba
    Data = pd.read_csv(f'query-Muestras-SQL.csv', sep=';')
    num_aps=Data['ID'].nunique()    
    ### Cambio fase4, conservar número de Beam y calculo numero muestras
    num_muestras_hora=math.ceil(num_aps*pc_muestra)
    Data['BEAM']=Data['BASE']
    ### fin Cambio fase4
    Data['BASE']='principal'
    print(num_aps,num_muestras_hora)

1515 455


## Lectura Archivos
###### Codigo para realizar el proceso con lectura de archivos

### Leer Archivo Festivos

In [7]:
ubicacionArchivoFestivos="F:/Workflow/Projects\MinTIC_SpeedTest/ProcessScripts/"
ubicacionArchivoFestivos="" #en caso de no ejecutar en el servidor
diasFestivos=pd.read_csv(f'{ubicacionArchivoFestivos}initialize-general-list-holidays.csv',sep=";")
diasFestivos['fecha']=pd.to_datetime(diasFestivos[['YEAR', 'MONTH', 'DAY']])

### Preprocesamiento

In [8]:
Data['FECHA']=pd.to_datetime(Data['FECHA']).dt.date
Data['FECHA']=pd.to_datetime(Data['FECHA'])
maxFecha=Data['FECHA'].max()
minFecha=maxFecha + datetime.timedelta(days=-14)
minFechaFDS=maxFecha + datetime.timedelta(days=-28)

In [9]:
Data['DIA_S']=Data['FECHA'].dt.dayofweek
Data['FILTRO']=Data['DIA_S'].isin([5,6])
Data['FILTRO']=np.where(Data['DIA_S'].isin([5,6]) , 'fds','hab')
Data['FILTRO']=np.where(Data['FECHA'].isin(diasFestivos['fecha']),'fds',Data['FILTRO'])

In [10]:
Data.drop_duplicates(subset=["ID","BANDA","SUBIDA_LIM","BAJADA_LIM","FECHA","HORA","CONDICION","VELOCIDAD_BAJADA","VELOCIDAD_SUBIDA","CUMPLE_BANDA","CUMPLE_SUBIDA","CUMPLE_BAJADA"],keep='first', inplace=True)
Data=Data[(Data['HORA']>=6)&(Data['HORA']<=20)]
DataTotal=Data.copy()
Data=Data[(Data['FECHA']>=minFecha)&(Data['FECHA']<maxFecha)]

##  Resumenes validación

In [11]:
def pieChart(col):
    A = col.value_counts()
    indices = A.index
    plt.pie(A, labels=indices, autopct="%0.0f %%", colors=['dodgerblue','red'])
    plt.axis("equal")
    plt.show()    

In [12]:
## TORTA 1
Data[Data['BASE']=='principal']['CONDICION'].value_counts()

E    86525
F    24350
Name: CONDICION, dtype: int64

In [13]:
Data[Data['BASE']=='principal']['CONDICION'].value_counts()/len(Data[Data['BASE']=='principal'])

E    0.780383
F    0.219617
Name: CONDICION, dtype: float64

In [14]:
if con_graficas:
    pieChart(Data[Data['BASE']=='principal']['CONDICION'])

In [15]:
resumen_exito_beam=pd.pivot_table(Data[Data['BASE']=='principal'][['BEAM','CONDICION']].value_counts().to_frame().reset_index(), values=0, index=['BEAM'], columns=['CONDICION'], aggfunc=np.sum)
resumen_exito_beam

CONDICION,E,F
BEAM,,
-1,64833,18673
2,1930,396
3,4639,1088
4,4748,1424
5,1914,421
6,216,9
7,395,173
8,4169,1316
9,1996,436


In [16]:
# Fase 4,  grafico stack por beam
if con_graficas:
    f, ax = plt.subplots(1,2, figsize=(10,4))
    resumen_exito_beam.plot.bar(stacked=True, ax=ax[0])
    resumen_exito_beam.div(resumen_exito_beam.sum(axis=1), axis=0).plot.bar(stacked=True, ax=ax[1])

In [17]:
DataExito=Data[Data['CONDICION']=='E']

In [18]:
## TORTA 2
DataExito[DataExito['BASE']=='principal']['CUMPLE_BANDA'].value_counts()

SI    75977
NO    10548
Name: CUMPLE_BANDA, dtype: int64

In [19]:
DataExito[DataExito['BASE']=='principal']['CUMPLE_BANDA'].value_counts()/len(DataExito)

SI    0.878093
NO    0.121907
Name: CUMPLE_BANDA, dtype: float64

In [20]:
if con_graficas:
    pieChart(DataExito[DataExito['BASE']=='principal']['CUMPLE_BANDA'])

In [21]:
resumen_cumple_beam=pd.pivot_table(DataExito[DataExito['BASE']=='principal'][['BEAM','CUMPLE_BANDA']].value_counts().to_frame().reset_index(), values=0, index=['BEAM'], columns=['CUMPLE_BANDA'], aggfunc=np.sum)
resumen_cumple_beam

CUMPLE_BANDA,NO,SI
BEAM,,
-1,8061,56772
2,242,1688
3,481,4158
4,409,4339
5,105,1809
6,43,173
7,24,371
8,829,3340
9,130,1866


In [22]:
# Fase 4,  grafico stack por beam
if con_graficas:
    f, ax = plt.subplots(1,2, figsize=(10,4))
    resumen_cumple_beam.plot.bar(stacked=True, ax=ax[0])
    resumen_cumple_beam.div(resumen_cumple_beam.sum(axis=1), axis=0).plot.bar(stacked=True, ax=ax[1])

In [23]:
DataExito[DataExito['BASE']=='principal']['CUMPLE_SUBIDA'].value_counts()/len(DataExito[DataExito['BASE']=='principal'])

SI    0.898018
NO    0.101982
Name: CUMPLE_SUBIDA, dtype: float64

In [24]:
if con_graficas:
    resumen_cumple_beam=pd.pivot_table(DataExito[DataExito['BASE']=='principal'][['BEAM','CUMPLE_SUBIDA']].value_counts().to_frame().reset_index(), values=0, index=['BEAM'], columns=['CUMPLE_SUBIDA'], aggfunc=np.sum)
    resumen_cumple_beam
    f, ax = plt.subplots(1,2, figsize=(10,4))
    resumen_cumple_beam.plot.bar(stacked=True, ax=ax[0])
    resumen_cumple_beam.div(resumen_cumple_beam.sum(axis=1), axis=0).plot.bar(stacked=True, ax=ax[1])

In [25]:
DataExito[DataExito['BASE']=='principal']['CUMPLE_BAJADA'].value_counts()/len(DataExito[DataExito['BASE']=='principal'])

SI    0.967189
NO    0.032811
Name: CUMPLE_BAJADA, dtype: float64

In [26]:
if con_graficas:
    resumen_cumple_beam=pd.pivot_table(DataExito[DataExito['BASE']=='principal'][['BEAM','CUMPLE_BAJADA']].value_counts().to_frame().reset_index(), values=0, index=['BEAM'], columns=['CUMPLE_BAJADA'], aggfunc=np.sum)
    resumen_cumple_beam
    f, ax = plt.subplots(1,2, figsize=(10,4))
    resumen_cumple_beam.plot.bar(stacked=True, ax=ax[0])
    resumen_cumple_beam.div(resumen_cumple_beam.sum(axis=1), axis=0).plot.bar(stacked=True, ax=ax[1])

In [27]:
#conteo centros con exito
DataExito[DataExito['BASE']=='principal']['ID'].nunique()

1378

In [28]:
#conteo centrol críticos
num_aps-DataExito[DataExito['BASE']=='principal']['ID'].nunique()

137

In [29]:
###  GRAFICO

In [30]:
res_hora_cumple_Banda=pd.crosstab(index=DataExito['HORA'],
            columns=DataExito['CUMPLE_BANDA'], margins=True)

In [31]:
res_hora_cumple_Banda['NO']=res_hora_cumple_Banda['NO']/res_hora_cumple_Banda['All']
res_hora_cumple_Banda['SI']=res_hora_cumple_Banda['SI']/res_hora_cumple_Banda['All']

In [32]:
pd.crosstab(index=DataExito['HORA'],
            columns=DataExito['CUMPLE_BANDA'], margins=True, normalize='index')

CUMPLE_BANDA,NO,SI
HORA,,
6,0.099698,0.900302
7,0.098231,0.901769
8,0.108386,0.891614
9,0.122389,0.877611
10,0.129474,0.870526
11,0.130810,0.869190
12,0.137399,0.862601
13,0.129760,0.870240
14,0.133605,0.866395


#### Gráfico

In [33]:
if con_graficas:
    res_grafico=pd.pivot_table(Data[Data['BASE']=='principal'].groupby(['BANDA','HORA','CUMPLE_BANDA'])['ID'].count().reset_index(), index=['BANDA','HORA'],columns=['CUMPLE_BANDA'], values='ID', aggfunc=np.sum, fill_value=0)
    res_grafico['TOTAL']=res_grafico.sum(axis=1)
    res_grafico['PORCF']=res_grafico['SI']/res_grafico['TOTAL']
    res_grafico.reset_index(inplace=True)
    if con_graficas:
        fig, ax = plt.subplots()
        for label, grp in res_grafico.groupby('BANDA'):
            grp.plot(x = 'HORA', y = 'PORCF',ax = ax, label = label)
    res_grafico.to_csv('grafica.csv')

In [34]:
if con_graficas:
    res_grafico=pd.pivot_table(DataExito[DataExito['BASE']=='principal'].groupby(['BANDA','HORA','CUMPLE_BANDA'])['ID'].count().reset_index(), index=['BANDA','HORA'],columns=['CUMPLE_BANDA'], values='ID', aggfunc=np.sum, fill_value=0)
    res_grafico['TOTAL']=res_grafico.sum(axis=1)
    res_grafico['PORCF']=res_grafico['SI']/res_grafico['TOTAL']
    res_grafico.reset_index(inplace=True)
    if con_graficas:
        fig, ax = plt.subplots()
        for label, grp in res_grafico.groupby('BANDA'):
            grp.plot(x = 'HORA', y = 'PORCF',ax = ax, label = label)
    res_grafico.to_csv('grafica.csv')

In [35]:
def quantiles_adaptados(x):
    conteo=x.count()
    if conteo<=5:
        return x.quantile(0.5)
    elif conteo<=10:
        return x.quantile(0.2)
    else:
        return x.quantile(0.1)

### Archivos de Resumen

In [36]:
today = date.today()
D=today.day
M=today.month
Y=today.year

# Habiles

In [37]:
Data_Habil= Data[(Data['BASE']=='principal')&(Data['FILTRO']=='hab')].groupby(['ID','HORA']).agg(
                CONTEO=('ID','count'),
                SUBIDA=('VELOCIDAD_SUBIDA', quantiles_adaptados),
                BAJADA=('VELOCIDAD_BAJADA', quantiles_adaptados)
            )

In [38]:
Data_Habil.reset_index(inplace=True)
BANDA=Data[['ID','BANDA','SUBIDA_LIM','BAJADA_LIM']].drop_duplicates(subset='ID',keep='last')
Data_Habil=Data_Habil.merge(BANDA,left_on='ID',right_on='ID', how='left')
Data_Habil.rename(columns={'BAJADA_LIM':'BAJA','SUBIDA_LIM':'SUBE','BAJADA':'BAJA_R','SUBIDA':'SUBE_R'}, inplace=True)
Data_Habil=Data_Habil[['ID','BANDA','HORA','BAJA','SUBE','BAJA_R','SUBE_R']]

In [39]:
Data_Habil['CUMPLE_BAJA']=np.where(Data_Habil['BAJA_R']>=Data_Habil['BAJA'],'SI','NO')
Data_Habil['CUMPLE_SUBE']=np.where(Data_Habil['SUBE_R']>=Data_Habil['SUBE'],'SI','NO')
Data_Habil['CUMPLE']=np.where( (Data_Habil['CUMPLE_BAJA']=='SI') & (Data_Habil['CUMPLE_SUBE']=='SI'),'SI','NO')
Data_Habil['PORC_BAJADA']=Data_Habil['BAJA_R']/Data_Habil['BAJA']
Data_Habil['PORC_SUBE']=Data_Habil['SUBE_R']/Data_Habil['SUBE']
Data_Habil['CERCANO_BAJA']=np.where(Data_Habil['PORC_BAJADA']>1, 'CUMPLE', np.where(Data_Habil['PORC_BAJADA']>0.9,'SI','NO'))
Data_Habil['CERCANO_SUBE']=np.where(Data_Habil['PORC_SUBE']>1, 'CUMPLE', np.where(Data_Habil['PORC_SUBE']>0.9,'SI','NO'))
Data_Habil['CERCANO']=np.where( (Data_Habil['CERCANO_BAJA']=='CUMPLE') & (Data_Habil['CERCANO_SUBE']=='CUMPLE'),'CUMPLE',
                                 np.where((Data_Habil['CERCANO_BAJA']=='SI') & (Data_Habil['CERCANO_SUBE']=='SI'),'AMBOS',
                                         np.where((Data_Habil['CERCANO_BAJA']=='CUMPLE') & (Data_Habil['CERCANO_SUBE']=='SI'),'SUBE',
                                                 np.where((Data_Habil['CERCANO_BAJA']=='SI') & (Data_Habil['CERCANO_SUBE']=='CUMPLE'),'BAJA','NO'))))

In [40]:
Data_Habil_Res=Data_Habil.copy()
Data_Habil_Res['TIPO2']=np.where(Data_Habil_Res['CERCANO']=="NO",'NO','SI')
Data_Habil_Res=pd.pivot_table(Data_Habil_Res[['ID','BANDA','CUMPLE']], index=['ID','BANDA'], columns=['CUMPLE'], aggfunc=len, fill_value=0)
Data_Habil_Res['PORC']=Data_Habil_Res['NO']/(Data_Habil_Res['NO']+Data_Habil_Res['SI'])
Data_Habil_Res['TIPO']= np.where(Data_Habil_Res['PORC']==1,'CRITICO',np.where(Data_Habil_Res['PORC']==0,'BUENO','REGULAR'))

In [41]:
if con_graficas:
    Data_Habil.head(14).style.set_table_styles([{
        'selector':'th', 'props':[('background-color','#c00000'),('color','white')]
    }])

In [42]:
### Guardar Achivos
Data_Habil.to_csv('RESUMEN_CENTROS_HORA_'+str(Y*10000+M*100+D)+'_HABIL.csv',sep=';', index=False)
Data_Habil_Res.to_csv('RESUMEN_CENTROS_'+str(Y*10000+M*100+D)+'_HABIL.csv',sep=';', index=False)

### FDS

In [43]:
Data_fds= DataTotal[(DataTotal['BASE']=='principal')&(DataTotal['FILTRO']=='fds')&(DataTotal['FECHA']>=minFechaFDS)&(DataTotal['FECHA']<maxFecha)].groupby(['ID','HORA']).agg(
                CONTEO=('ID','count'),
                SUBIDA=('VELOCIDAD_SUBIDA', quantiles_adaptados),
                BAJADA=('VELOCIDAD_BAJADA', quantiles_adaptados)
            )

In [44]:
Data_fds.reset_index(inplace=True)
BANDA=Data[['ID','BANDA','SUBIDA_LIM','BAJADA_LIM']].drop_duplicates(subset='ID',keep='last')
Data_fds=Data_fds.merge(BANDA,left_on='ID',right_on='ID', how='left')
Data_fds.rename(columns={'BAJADA_LIM':'BAJA','SUBIDA_LIM':'SUBE','BAJADA':'BAJA_R','SUBIDA':'SUBE_R'}, inplace=True)
Data_fds=Data_fds[['ID','BANDA','HORA','BAJA','SUBE','BAJA_R','SUBE_R']]

In [45]:
Data_fds['CUMPLE_BAJA']=np.where(Data_fds['BAJA_R']>=Data_fds['BAJA'],'SI','NO')
Data_fds['CUMPLE_SUBE']=np.where(Data_fds['SUBE_R']>=Data_fds['SUBE'],'SI','NO')
Data_fds['CUMPLE']=np.where( (Data_fds['CUMPLE_BAJA']=='SI') & (Data_fds['CUMPLE_SUBE']=='SI'),'SI','NO')
Data_fds['PORC_BAJADA']=Data_fds['BAJA_R']/Data_fds['BAJA']
Data_fds['PORC_SUBE']=Data_fds['SUBE_R']/Data_fds['SUBE']
Data_fds['CERCANO_BAJA']=np.where(Data_fds['PORC_BAJADA']>1, 'CUMPLE', np.where(Data_fds['PORC_BAJADA']>0.9,'SI','NO'))
Data_fds['CERCANO_SUBE']=np.where(Data_fds['PORC_SUBE']>1, 'CUMPLE', np.where(Data_fds['PORC_SUBE']>0.9,'SI','NO'))
Data_fds['CERCANO']=np.where( (Data_fds['CERCANO_BAJA']=='CUMPLE') & (Data_fds['CERCANO_SUBE']=='CUMPLE'),'CUMPLE',
                                 np.where((Data_fds['CERCANO_BAJA']=='SI') & (Data_fds['CERCANO_SUBE']=='SI'),'AMBOS',
                                         np.where((Data_fds['CERCANO_BAJA']=='CUMPLE') & (Data_fds['CERCANO_SUBE']=='SI'),'SUBE',
                                                 np.where((Data_fds['CERCANO_BAJA']=='SI') & (Data_fds['CERCANO_SUBE']=='CUMPLE'),'BAJA','NO'))))

In [46]:
Data_fds_Res=Data_fds.copy()
Data_fds_Res['TIPO2']=np.where(Data_fds_Res['CERCANO']=="NO",'NO','SI')
Data_fds_Res=pd.pivot_table(Data_fds_Res[['ID','BANDA','CUMPLE']], index=['ID','BANDA'], columns=['CUMPLE'], aggfunc=len, fill_value=0)
Data_fds_Res['PORC']=Data_fds_Res['NO']/(Data_fds_Res['NO']+Data_fds_Res['SI'])
Data_fds_Res['TIPO']= np.where(Data_fds_Res['PORC']==1,'CRITICO',np.where(Data_fds_Res['PORC']==0,'BUENO','REGULAR'))

In [47]:
Data_fds.to_csv('RESUMEN_CENTROS_HORA_'+str(Y*10000+M*100+D)+'_FDS.csv',sep=';',index=False)
Data_fds_Res.to_csv('RESUMEN_CENTROS_'+str(Y*10000+M*100+D)+'_FDS.csv',sep=';',index=False)

# Maestro Aps

In [48]:
maestro_Aps=Data[['ID','BANDA','BEAM','FASE']].drop_duplicates()

In [49]:
maestro_Aps['BEAM'].value_counts()

-1     1086
 4       99
 8       91
 3       89
 2       38
 5       35
 10      34
 9       30
 7        9
 6        4
Name: BEAM, dtype: int64

## Funcion Asignacion

In [50]:
def asignarMuestras(Data,num_muestras_hora):
    DataExito=Data[Data['CONDICION']=='E']    
    #preprocesamiento
    DataExito['IND']=((DataExito['VELOCIDAD_SUBIDA']-DataExito['SUBIDA_LIM'])+(DataExito['VELOCIDAD_BAJADA']-DataExito['BAJADA_LIM']))/2
    
    tabla_ind=DataExito.groupby(['ID','HORA'])[['IND']].median()
    tabla_ind.reset_index(inplace=True)
    tabla_freq=DataExito.groupby(['ID','HORA'])[['CONDICION']].count().reset_index()

    #calculo cumplimiento para mejores y peores casos
    tabla_ind_cump=pd.pivot_table(DataExito[['ID','HORA','CUMPLE_BANDA','CONDICION']], index=['ID','HORA'], columns=['CUMPLE_BANDA'], aggfunc=len, fill_value=0)
    tabla_ind_cump.columns=tabla_ind_cump.columns.droplevel()
    tabla_ind_cump.reset_index(inplace=True)
    tabla_ind_cump['ind_cump']=tabla_ind_cump['SI']/(tabla_ind_cump['SI']+tabla_ind_cump['NO'])
    
    tablaBase=tabla_ind.merge(tabla_freq, left_on=['ID','HORA'], right_on=['ID','HORA'], how='left')
    tablaBase=tablaBase.merge(tabla_ind_cump[['ID','HORA','ind_cump']], left_on=['ID','HORA'], right_on=['ID','HORA'], how='left')
    
    tablaBase.columns=['ID','HORA','IND','FREQ','ind_cump']
    tablaBase.reset_index(inplace=True)
    
    tablaBase_ori=tablaBase.copy()
    
    #Distribución aps bajos
    tablaBase.sort_values(['ID','IND','FREQ'], ascending=False, inplace=True)
    tablaBase.reset_index(drop=True, inplace=True)
    tabla_primera_opcion=tablaBase.groupby(['ID']).nth(0)
    casos_bajos=tabla_primera_opcion[tabla_primera_opcion['IND']<0]
    muestra_bajas_hora=math.ceil(len(casos_bajos)/num_horas)

    casos_bajos=casos_bajos.sort_values('IND')
    casos_bajos.reset_index(inplace=True)
    casos_bajos.reset_index(inplace=True)
    #casos_bajos
    
    #lograr la mayor distribución de hora posible
    casos_bajos['franja']=np.where(casos_bajos['level_0']//num_horas%2==0,casos_bajos['level_0']%num_horas,(num_horas-1-casos_bajos['level_0']%num_horas))
    casos_bajos['hora']=casos_bajos['franja']+6
    tablaBase=tablaBase[~tablaBase['ID'].isin(casos_bajos['ID'].unique())]

    num_muestras_hora=num_muestras_hora-muestra_bajas_hora
    asignacionFinal=pd.DataFrame()

    #Asignacion Principal
    i=0
    num_horas_sobre=1
    while num_horas_sobre > 0:   
        #Reordenar la base
        tablaBase.sort_values(['ID','IND','FREQ'], ascending=False, inplace=True)
        tablaBase.reset_index(drop=True, inplace=True)
        tablaBase['index']=tablaBase.index

        tablaBase['dif']=tablaBase.groupby('ID')['IND'].diff(-1)
        tablaBase['index_N']=tablaBase.groupby('ID')['index'].shift(periods=-1)
        tablaBase['HORA_N']=tablaBase.groupby('ID')['HORA'].shift(periods=-1)
        tablaBase['IND_N']=tablaBase.groupby('ID')['IND'].shift(periods=-1)
        tablaBase['FREQ_N']=tablaBase.groupby('ID')['FREQ'].shift(periods=-1)

        tabla_primera_opcion=tablaBase.groupby(['ID']).nth(0)
        tabla_primera_opcion.reset_index(inplace=True)

        tabla_ronda=tabla_primera_opcion

        resumen=tabla_ronda[['HORA']].value_counts().to_frame()
        resumen['DIF']=resumen[0]-num_muestras_hora

        horas_sobre=resumen[resumen['DIF']>0]           
        num_horas_sobre=len(horas_sobre)

        # guardar valores
        if num_horas_sobre>0:
            hora_aux=horas_sobre.index[0][0]
            dif_aux=horas_sobre.iloc[0]['DIF']
            tabla_ronda=tabla_ronda.sort_values('dif')
            saldo=tabla_ronda[tabla_ronda['HORA']==hora_aux][dif_aux:]        
            if i==0:
                asignacionFinal=saldo
            else: 
                asignacionFinal=asignacionFinal.append(saldo)            

            tablaBase=tablaBase[tablaBase['HORA']!=hora_aux]
            tablaBase=tablaBase[~tablaBase['ID'].isin(saldo['ID'])]            
            i+=1

    if i==0:
        asignacionFinal=tabla_ronda
    else: 
        asignacionFinal=asignacionFinal.append(tabla_ronda)


    # Matriz de posiciciones a llenar
    num_muestras_hora=num_muestras_hora+muestra_bajas_hora
    tabla_final=pd.DataFrame({'hora':list(range(6,20+1))},columns=['hora']+list(range(num_muestras_hora)))
    tabla_final.set_index('hora',inplace=True)
    tabla_final['pos']=0

    #Matriz de causa asignacion para asignación de satelitales
    tabla_final_causa=pd.DataFrame({'hora':list(range(6,20+1))},columns=['hora']+list(range(num_muestras_hora)))
    tabla_final_causa.set_index('hora',inplace=True)
    
    # Asignar resultado anterior
    for index, row in asignacionFinal.iterrows():    
        hora_aux=int(row['HORA'])
        pos=tabla_final.at[hora_aux,'pos']    
        tabla_final.at[hora_aux,pos]=int(row['ID'])    
        tabla_final.at[hora_aux,'pos']=tabla_final.at[hora_aux,'pos']+1
        tabla_final_causa.at[hora_aux,pos]='1.m_aps'
    
    #Asignar casos bajos
    for ind,row in casos_bajos.iterrows():
        t_aux=tabla_final.copy()
        t_aux=t_aux[(t_aux['pos']<num_muestras_hora)]
        sel_hora=row['hora']
        idx=int(row['ID'])
        pos=tabla_final.at[sel_hora,'pos']
        tabla_final.at[sel_hora,pos]=idx    
        tabla_final.at[sel_hora,'pos']=tabla_final.at[sel_hora,'pos']+1
        tabla_final_causa.at[sel_hora,pos]='2.bajos'
    
    #Validar aps unicos
    ##print(tabla_final.reset_index().drop(columns=['pos']).melt(id_vars=['hora'],var_name='muestra',value_name='ID').nunique())
    #Agregar 100% F
    id_asignados=list(casos_bajos['ID'].unique())+list(asignacionFinal['ID'].unique())
    APs_sin_Exito=Data[~Data['ID'].isin(id_asignados)]
    APs_sin_Exito=list(APs_sin_Exito['ID'].unique())
    
    for idx in APs_sin_Exito:
        t_aux=tabla_final.copy()
        t_aux=t_aux[(t_aux['pos']<num_muestras_hora)]
        t_aux=t_aux.sort_values('pos',ascending=True)
        sel_hora=t_aux.head(1).index[0]
        pos=tabla_final.at[sel_hora,'pos']
        tabla_final.at[sel_hora,pos]=idx    
        tabla_final.at[sel_hora,'pos']=tabla_final.at[sel_hora,'pos']+1
        tabla_final_causa.at[sel_hora,pos]='3.sin_E'
    
    #Asignar mejores casos
    tablaBase=tabla_ind.merge(tabla_freq, left_on=['ID','HORA'], right_on=['ID','HORA'], how='left')    
    tablaBase=tablaBase.merge(tabla_ind_cump[['ID','HORA','ind_cump']], left_on=['ID','HORA'], right_on=['ID','HORA'], how='left')
    tablaBase.columns=['ID','HORA','IND','FREQ','ind_cump']
    tablaBase.reset_index(inplace=True)    
    mejores_hora=tablaBase[(tablaBase['ind_cump']==1)&(tablaBase['FREQ']>=3)]
    group_casos_mejora=mejores_hora.groupby('ID')

    i=0
    for g in group_casos_mejora.groups:
        i=i+1
        group = group_casos_mejora.get_group(g)
        val_ids=group['HORA'].values
        t_aux=tabla_final.copy()
        t_aux=t_aux[(t_aux['pos']<num_muestras_hora)&(t_aux.index.isin(val_ids))]
        t_aux= t_aux[~t_aux.isin([g]).any(axis=1)]

        for sel_hora in t_aux.index:
            pos=tabla_final.at[sel_hora,'pos']
            tabla_final.at[sel_hora,pos]=int(g)  
            tabla_final.at[sel_hora,'pos']=tabla_final.at[sel_hora,'pos']+1
            tabla_final_causa.at[sel_hora,pos]='4.mc_hora'
    
    #Completar faltantes con mejores casos
    tabla_ind_total=pd.pivot_table(DataExito[['ID','CUMPLE_BANDA','CONDICION']], index=['ID'], columns=['CUMPLE_BANDA'], aggfunc=len, fill_value=0)
    tabla_ind_total.columns=tabla_ind_total.columns.droplevel()
    tabla_ind_total.reset_index(inplace=True)
    tabla_ind_total['ind']=tabla_ind_total['SI']/(tabla_ind_total['SI']+tabla_ind_total['NO'])
    tabla_ind_total=tabla_ind_total.sort_values(['ind','SI'], ascending=False)
    pos_mejores=0
    resultado_mejores_casos=tabla_ind_total[tabla_ind_total['ind']==1].merge(maestro_Aps, left_on='ID', right_on='ID', how='left')

    while len(tabla_final[tabla_final['pos']<num_muestras_hora])>0:
        t_aux=tabla_final[tabla_final['pos']<num_muestras_hora]
        id_aux=tabla_ind_total['ID'].values[pos_mejores]
        t_aux= t_aux[~t_aux.isin([id_aux]).any(axis=1)]

        for sel_hora in t_aux.index:
            pos=tabla_final.at[sel_hora,'pos']
            tabla_final.at[sel_hora,pos]=int(id_aux)  
            tabla_final.at[sel_hora,'pos']=tabla_final.at[sel_hora,'pos']+1
            tabla_final_causa.at[sel_hora,pos]='5.mc'
        pos_mejores=pos_mejores+1

    ## validar duplicados por hora
    for hora,row in tabla_final.iterrows():    
        a_set = set(row)
        contains_duplicates = len(row) != len(a_set)
        print(hora,contains_duplicates)
    
    #Alistamiento de salida
    tabla_ind_total['tipo']=np.where(tabla_ind_total['ind']==0, 'CRITICO', np.where(tabla_ind_total['ind']==1,'BUENO','REGULAR'))
    tabla_final_un=tabla_final.reset_index().drop(columns=['pos']).melt(id_vars=['hora'],var_name='muestra',value_name='ID')
    tabla_final_un=tabla_final_un.merge(tabla_ind_total, left_on='ID', right_on='ID', how='left')
    tabla_final_un['tipo']=np.where(tabla_final_un['tipo'].isnull(),'CRITICO',tabla_final_un['tipo'])
    tabla_final_un=tabla_final_un.merge(maestro_Aps, left_on='ID', right_on='ID', how='left')
    # Agregar indicador por horas
    tabla_final_un=tabla_final_un.merge(tablaBase_ori, left_on=['ID','hora'], right_on=['ID','HORA'], how='left')
    
    #agregar causas
    tabla_final_un_causas=tabla_final_causa.reset_index().melt(id_vars=['hora'],var_name='muestra',value_name='causa')
    tabla_final_un=tabla_final_un.merge(tabla_final_un_causas, left_on=['hora','muestra'], right_on=['hora','muestra'], how='left')
    #fin de la funcion,  retorno
    return tabla_final_un,resultado_mejores_casos

In [51]:
def validarSatelitales(tablaBase,tablaMejores):
    tabla_fina_com=tablaBase.copy()
    mc_fina_com=tablaMejores.copy()
    tabla_fina_com.reset_index(drop=True,inplace=True)
    #Resultado número de muestras por horas de satelitales  (beams diferente a -1)
    df_muestras=tabla_fina_com[tabla_fina_com['BEAM']!=-1][['hora','BEAM']].value_counts().to_frame().reset_index()
    df_muestras=df_muestras[df_muestras[0]>n_beams_sat]    
    mc_fina_com=mc_fina_com[mc_fina_com['BEAM']==-1]
    
    res_mc=mc_fina_com[['BANDA','FASE']].value_counts().to_frame().reset_index()
    # Iteración
    for key,row in df_muestras.iterrows():
        (aux_hora,aux_beam,aux_num_casos)=row.values
        ind_rev=tabla_fina_com[tabla_fina_com['hora']==aux_hora]['ID'].values
        df_banda_hora_sobre=tabla_fina_com[(tabla_fina_com['hora']==aux_hora)&(tabla_fina_com['BEAM']==aux_beam)].sort_values('causa', ascending=False)
        exceso=aux_num_casos-n_beams_sat
        df_exceso=df_banda_hora_sobre[(df_banda_hora_sobre['causa']=='4.mc_hora')|(df_banda_hora_sobre['causa']=='5.mc')][:exceso]
        res_exceso=df_exceso[['BANDA','FASE']].value_counts().to_frame().reset_index()
        reemplazos=pd.DataFrame(columns=mc_fina_com.columns)
        for key,row in res_exceso.iterrows():    
            df_bolsa_reemplazos=mc_fina_com[(mc_fina_com['BANDA']==row['BANDA']) & (mc_fina_com['FASE']==row['FASE'])]
            df_bolsa_reemplazos=df_bolsa_reemplazos[~df_bolsa_reemplazos['ID'].isin(ind_rev)]    
            res_mc_rem=df_bolsa_reemplazos[['BANDA','FASE']].value_counts().to_frame().reset_index()    
            num_bolsa=len(df_bolsa_reemplazos)
            reemplazos=reemplazos.append(df_bolsa_reemplazos.sample(min(row[0],num_bolsa)))
        reemplazos['hora']=aux_hora
        reemplazos['causa']='5.mc'
        tabla_fina_com=tabla_fina_com[~tabla_fina_com.index.isin(df_exceso.index)]
        tabla_fina_com=tabla_fina_com.append(reemplazos)
        tabla_fina_com.reset_index(drop=True, inplace=True)
    return tabla_fina_com

###  Distribucion de las muestras en las bandas

In [52]:
distribucion=Data[['ID','BANDA','FASE']].drop_duplicates()
res_dist=distribucion[['BANDA','FASE']].value_counts().to_frame()
res_dist['tamMuestra']=res_dist[0]*pc_muestra
res_dist['tamMuestra_int']=res_dist['tamMuestra'].apply(np.ceil).astype(int)
res_dist['tamMuestra_int'].sum()
res_dist.head(1).index[0]

('DL:12.00Mb-UL:3.00Mb', 11)

In [53]:
(num_muestras_hora-res_dist['tamMuestra_int'].sum())

-4

In [54]:
num_muestras_hora

455

In [55]:
## Ajustar tamaño
res_dist.at[res_dist.head(1).index,'tamMuestra_int']=res_dist.at[res_dist.head(1).index[0],'tamMuestra_int']+(num_muestras_hora-res_dist['tamMuestra_int'].sum())

In [56]:
res_dist['tamMuestra_int'].sum()

455

In [57]:
res_dist.reset_index(inplace=True)

In [58]:
res_dist['minimo']=res_dist[0]/num_horas

In [59]:
res_dist

,BANDA,FASE,0,tamMuestra,tamMuestra_int,minimo
0,DL:12.00Mb-UL:3.00Mb,11,389,116.7,113,25.933333
1,DL:12.00Mb-UL:3.00Mb,12,337,101.1,102,22.466667
2,DL:15.00Mb-UL:3.75Mb,12,215,64.5,65,14.333333
3,DL:15.00Mb-UL:3.75Mb,11,199,59.7,60,13.266667
4,DL:18.00Mb-UL:4.50Mb,12,134,40.2,41,8.933333
5,DL:18.00Mb-UL:4.50Mb,11,119,35.7,36,7.933333
6,DL:21.00Mb-UL:5.25Mb,12,71,21.3,22,4.733333
7,DL:21.00Mb-UL:5.25Mb,11,51,15.3,16,3.400000


### Habiles

In [60]:
tabla_fina_com=pd.DataFrame()
mc_fina_com=pd.DataFrame()
i=0
for key,row in res_dist.iterrows():
    print(row['BANDA'],row['FASE'])
    (tabla_final_un,resultado_mejores_casos)=asignarMuestras(Data[(Data['FILTRO']=='hab')&(Data['BANDA']==row['BANDA'])&(Data['FASE']==row['FASE'])],row['tamMuestra_int'])    
    if i==0:
        tabla_fina_com=tabla_final_un.copy()
        mc_fina_com=resultado_mejores_casos.copy()
    else :
        tabla_fina_com=tabla_fina_com.append(tabla_final_un)
        mc_fina_com=mc_fina_com.append(resultado_mejores_casos)        
    i=i+1

DL:12.00Mb-UL:3.00Mb 11
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
DL:12.00Mb-UL:3.00Mb 12
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
DL:15.00Mb-UL:3.75Mb 12
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
DL:15.00Mb-UL:3.75Mb 11
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
DL:18.00Mb-UL:4.50Mb 12
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
DL:18.00Mb-UL:4.50Mb 11
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
DL:21.00Mb-UL:5.25Mb 12
6 False
7 False
8 False
9 False
10 False
11 Fa

In [61]:
sum(tabla_fina_com[['hora','ID']].value_counts()>1)

0

In [62]:
tabla_fina_com['ID'].nunique()

1515

In [63]:
pd.crosstab(index=tabla_fina_com['hora'], columns=tabla_fina_com['BEAM'], margins=True)

BEAM,-1,2,3,4,5,6,7,8,9,10,All
hora,,,,,,,,,,,
6,367,7,15,15,7,2,1,29,4,8,455
7,342,11,20,30,10,0,2,22,9,9,455
8,342,8,22,25,7,0,2,35,8,6,455
9,362,6,16,22,10,0,1,21,10,7,455
10,364,5,16,22,9,0,2,18,11,8,455
11,345,6,24,25,14,1,2,19,12,7,455
12,320,10,33,29,13,1,2,21,14,12,455
13,336,9,30,26,13,1,6,13,9,12,455
14,329,14,29,34,12,1,5,13,12,6,455


In [64]:
tabla_fina_com.groupby('BEAM')['ID'].nunique()

BEAM
-1     1086
 2       38
 3       89
 4       99
 5       35
 6        4
 7        9
 8       91
 9       30
 10      34
Name: ID, dtype: int64

In [65]:
tabla_fina_com=validarSatelitales(tabla_fina_com,mc_fina_com)

In [66]:
sum(tabla_fina_com[['hora','ID']].value_counts()>1)

0

In [67]:
tabla_fina_com.groupby('BEAM')['ID'].nunique()

BEAM
-1     1086
 2       38
 3       89
 4       99
 5       35
 6        4
 7        9
 8       91
 9       30
 10      34
Name: ID, dtype: int64

In [68]:
pd.crosstab(index=tabla_fina_com['hora'],
            columns=tabla_fina_com['tipo'], margins=True)

tipo,BUENO,CRITICO,REGULAR,All
hora,,,,
6,140,10,294,444
7,189,6,242,437
8,206,8,214,428
9,216,27,205,448
10,226,29,196,451
11,227,18,196,441
12,225,26,175,426
13,223,17,195,435
14,222,6,200,428


In [69]:
pd.crosstab(index=tabla_fina_com['hora'], columns=tabla_fina_com['BEAM'], margins=True)

BEAM,-1,2,3,4,5,6,7,8,9,10,All
hora,,,,,,,,,,,
6,378,7,15,15,7,2,1,18,4,8,455
7,360,11,18,18,10,0,2,18,9,9,455
8,369,8,18,18,7,0,2,19,8,6,455
9,369,6,16,18,10,0,1,18,10,7,455
10,368,5,16,18,9,0,2,18,11,8,455
11,359,6,18,18,14,1,2,18,12,7,455
12,342,10,18,18,13,1,2,18,14,12,448
13,352,9,18,18,13,1,6,13,9,12,451
14,355,14,18,18,12,1,5,13,12,6,454


In [70]:
tabla_fina_com['causa'].value_counts()

4.mc_hora    4964
1.m_aps      1327
5.mc          322
3.sin_E       138
2.bajos        50
Name: causa, dtype: int64

In [71]:
tabla_fina_com['ID'].nunique()

1515

In [72]:
pd.crosstab(index=tabla_fina_com['hora'], columns=tabla_fina_com['FASE'], margins=True)

FASE,11,12,All
hora,,,
6,225,230,455
7,225,230,455
8,225,230,455
9,225,230,455
10,225,230,455
11,225,230,455
12,225,223,448
13,225,226,451
14,225,229,454


In [73]:
archivo=tabla_fina_com[['ID','BANDA','hora']]
archivo.columns=['ID','BANDA','HORA']
archivo=archivo.sort_values('HORA')

In [74]:
archivo.isnull().sum()

ID       0
BANDA    0
HORA     0
dtype: int64

In [75]:
duplicateRowsDF = archivo[archivo.duplicated()]
duplicateRowsDF

,ID,BANDA,HORA


In [76]:
archivo_hab=archivo.copy()

In [77]:
final_hab=archivo.copy()

In [78]:
final_hab['ID'].nunique()

1515

In [79]:
final_hab.groupby('BANDA')['ID'].nunique()

BANDA
DL:12.00Mb-UL:3.00Mb    726
DL:15.00Mb-UL:3.75Mb    414
DL:18.00Mb-UL:4.50Mb    253
DL:21.00Mb-UL:5.25Mb    122
Name: ID, dtype: int64

In [80]:
res_dist['tamMuestra_int'].sum()

455

 ### Festivos

In [81]:
tabla_fina_com=pd.DataFrame()
mc_fina_com=pd.DataFrame()
i=0
for key,row in res_dist.iterrows():
    print(row['BANDA'],row['FASE'])
    (tabla_final_un,resultado_mejores_casos)=asignarMuestras(Data[(Data['FILTRO']=='fds')&(Data['BANDA']==row['BANDA'])&(Data['FASE']==row['FASE'])],row['tamMuestra_int'])
    if i==0:
        tabla_fina_com=tabla_final_un.copy()
        mc_fina_com=resultado_mejores_casos.copy()
    else :
        tabla_fina_com=tabla_fina_com.append(tabla_final_un)
        mc_fina_com=mc_fina_com.append(resultado_mejores_casos) 
    i=i+1

DL:12.00Mb-UL:3.00Mb 11
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
DL:12.00Mb-UL:3.00Mb 12
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
DL:15.00Mb-UL:3.75Mb 12
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
DL:15.00Mb-UL:3.75Mb 11
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
DL:18.00Mb-UL:4.50Mb 12
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
DL:18.00Mb-UL:4.50Mb 11
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
DL:21.00Mb-UL:5.25Mb 12
6 False
7 False
8 False
9 False
10 False
11 Fa

In [82]:
tabla_fina_com=validarSatelitales(tabla_fina_com,mc_fina_com)

In [83]:
pd.crosstab(index=tabla_fina_com['hora'],
            columns=tabla_fina_com['tipo'], margins=True)

tipo,BUENO,CRITICO,REGULAR,All
hora,,,,
6,283,16,155,454
7,302,11,140,453
8,339,9,106,454
9,353,14,83,450
10,351,20,65,436
11,351,21,61,433
12,344,26,66,436
13,350,24,63,437
14,344,18,72,434


In [84]:
pd.crosstab(index=tabla_fina_com['hora'], columns=tabla_fina_com['BEAM'], margins=True)

BEAM,-1,2,3,4,5,6,7,8,9,10,All
hora,,,,,,,,,,,
6,383,5,13,10,8,0,1,18,7,10,455
7,381,3,18,18,6,1,1,14,11,2,455
8,375,3,14,17,9,0,1,18,15,3,455
9,374,7,13,18,8,0,3,15,11,6,455
10,384,8,18,18,6,0,4,4,12,1,455
11,376,8,18,18,7,0,3,7,15,3,455
12,384,6,18,18,8,0,1,7,10,3,455
13,375,7,18,18,7,1,1,10,14,4,455
14,370,12,18,18,14,1,1,8,11,2,455


In [85]:
pd.crosstab(index=tabla_fina_com['hora'], columns=tabla_fina_com['FASE'], margins=True)

FASE,11,12,All
hora,,,
6,225,230,455
7,225,230,455
8,225,230,455
9,225,230,455
10,225,230,455
11,225,230,455
12,225,230,455
13,225,230,455
14,225,230,455


In [86]:
sum(tabla_fina_com[['hora','ID']].value_counts()>1)

0

In [87]:
tabla_fina_com['causa'].value_counts()

5.mc         2768
4.mc_hora    2541
1.m_aps      1267
3.sin_E       202
2.bajos        46
Name: causa, dtype: int64

In [88]:
tabla_fina_com['ID'].nunique()

1515

In [89]:
archivo=tabla_fina_com[['ID','BANDA','hora']]
archivo.columns=['ID','BANDA','HORA']
archivo=archivo.sort_values('HORA')

In [90]:
archivo.isnull().sum()

ID       0
BANDA    0
HORA     0
dtype: int64

In [91]:
duplicateRowsDF = archivo[archivo.duplicated()]
duplicateRowsDF

,ID,BANDA,HORA


In [92]:
archivo_fds=archivo.copy()

In [93]:
final_fds=archivo.copy()

In [94]:
final_fds['ID'].nunique()

1515

In [95]:
final_hab.groupby('BANDA')['ID'].nunique()

BANDA
DL:12.00Mb-UL:3.00Mb    726
DL:15.00Mb-UL:3.75Mb    414
DL:18.00Mb-UL:4.50Mb    253
DL:21.00Mb-UL:5.25Mb    122
Name: ID, dtype: int64

## Guardar Archivos

In [96]:
archivo_hab.to_csv('initialize-general-list-hab.csv',sep=';', index=False)

In [97]:
archivo_fds.to_csv('initialize-general-list-fds.csv',sep=';', index=False)

# Prueba Aleatoria
* Realizar asignacion aleatoria
* validar indicadores principales
* calcular número de franjas cumplidas en los 2 escenarios

In [98]:
APs_id=DataTotal['ID'].unique()
len(APs_id)

1515

In [99]:
tabla_final=pd.DataFrame({'hora':list(range(6,20+1))},columns=['hora']+list(range(num_muestras_hora)))
tabla_final.set_index('hora',inplace=True)
tabla_final['pos']=0

### Union con muestreos

In [100]:
DataTotal_validacion=DataTotal.copy()

In [101]:
DataTotal_validacion=DataTotal_validacion.merge(final_fds.rename(columns={'ID':'ID_fds'}), left_on=['HORA','ID'], right_on=['HORA','ID_fds'],how='left')

In [102]:
DataTotal_validacion=DataTotal_validacion.merge(final_hab.rename(columns={'ID':'ID_hab'}), left_on=['HORA','ID'], right_on=['HORA','ID_hab'],how='left')

In [103]:
DataTotal_validacion[DataTotal_validacion['ID_hab'].notnull()]

,ID,BANDA_x,SUBIDA_LIM,BAJADA_LIM,FECHA,HORA,CONDICION,VELOCIDAD_BAJADA,VELOCIDAD_SUBIDA,CUMPLE_BANDA,...,CUMPLE_BAJADA,BASE,FASE,BEAM,DIA_S,FILTRO,ID_fds,BANDA_y,ID_hab,BANDA
0,21110,DL:18.00Mb-UL:4.50Mb,4500.0,18000.0,2022-01-03,6,E,23698.09,6560.73,SI,...,SI,principal,11,-1,0,hab,21110,DL:18.00Mb-UL:4.50Mb,21110,DL:18.00Mb-UL:4.50Mb
1,31066,DL:15.00Mb-UL:3.75Mb,3750.0,15000.0,2022-01-03,6,E,13733.45,8974.24,NO,...,SI,principal,11,-1,0,hab,NaN,NaN,31066,DL:15.00Mb-UL:3.75Mb
2,32556,DL:12.00Mb-UL:3.00Mb,3000.0,12000.0,2022-01-03,6,E,21094.42,5722.06,SI,...,SI,principal,11,-1,0,hab,NaN,NaN,32556,DL:12.00Mb-UL:3.00Mb
3,70175,DL:12.00Mb-UL:3.00Mb,3000.0,12000.0,2022-01-03,6,E,15900.11,7674.89,SI,...,SI,principal,11,-1,0,hab,NaN,NaN,70175,DL:12.00Mb-UL:3.00Mb
6,32451,DL:15.00Mb-UL:3.75Mb,3750.0,15000.0,2022-01-03,6,E,25193.51,5204.55,SI,...,SI,principal,11,-1,0,hab,NaN,NaN,32451,DL:15.00Mb-UL:3.75Mb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220297,32160,DL:15.00Mb-UL:3.75Mb,3750.0,15000.0,2022-02-01,17,E,17093.23,5226.76,SI,...,SI,principal,12,-1,1,hab,NaN,NaN,32160,DL:15.00Mb-UL:3.75Mb
220300,47504,DL:18.00Mb-UL:4.50Mb,4500.0,18000.0,2022-02-02,10,E,18966.28,24257.06,SI,...,SI,principal,12,-1,2,hab,47504,DL:18.00Mb-UL:4.50Mb,47504,DL:18.00Mb-UL:4.50Mb
220304,22522,DL:21.00Mb-UL:5.25Mb,5250.0,21000.0,2022-02-01,10,E,24793.35,10312.80,SI,...,SI,principal,12,-1,1,hab,22522,DL:21.00Mb-UL:5.25Mb,22522,DL:21.00Mb-UL:5.25Mb
220308,22522,DL:21.00Mb-UL:5.25Mb,5250.0,21000.0,2022-02-01,20,E,24940.93,10524.31,SI,...,SI,principal,12,-1,1,hab,22522,DL:21.00Mb-UL:5.25Mb,22522,DL:21.00Mb-UL:5.25Mb


In [104]:
DataTotal_validacion['ID_analitica']=np.where(DataTotal_validacion['FILTRO']=='hab',DataTotal_validacion['ID_hab'],DataTotal_validacion['ID_fds'])

### Resumen indicadores
*  No se paga por esta medida

In [105]:
Resultados_analitica=DataTotal_validacion[(DataTotal_validacion['ID_analitica'].notnull())]

In [106]:
Resultados_analitica['ID_analitica'].nunique()

1515

#### EXITO / FRACASO

In [107]:
if con_graficas:
    pieChart(Resultados_analitica['CONDICION'])

#### CUMPLE BANDA

In [108]:
if con_graficas:
    pieChart(Resultados_analitica[Resultados_analitica['CONDICION']=='E']['CUMPLE_BANDA'])

### Subida

In [109]:
Resultados_analitica[Resultados_analitica['CONDICION']=='E']['CUMPLE_SUBIDA'].value_counts()/len(Resultados_analitica[Resultados_analitica['CONDICION']=='E'])

SI    0.975023
NO    0.024977
Name: CUMPLE_SUBIDA, dtype: float64

In [110]:
if con_graficas:
    pieChart(Resultados_analitica[Resultados_analitica['CONDICION']=='E']['CUMPLE_SUBIDA'])

## Bajada

In [111]:
Resultados_analitica[Resultados_analitica['CONDICION']=='E']['CUMPLE_BAJADA'].value_counts()/len(Resultados_analitica[Resultados_analitica['CONDICION']=='E'])

SI    0.99154
NO    0.00846
Name: CUMPLE_BAJADA, dtype: float64

In [112]:
if con_graficas:
    pieChart(Resultados_analitica[Resultados_analitica['CONDICION']=='E']['CUMPLE_BAJADA'])

In [113]:
Resultados_analitica['BASE'].value_counts()/len(Resultados_analitica)

principal    1.0
Name: BASE, dtype: float64

## Cumplimiento Franjas

In [114]:
dic_cumplimiento={'DL:12.00Mb-UL:3.00Mb':[12000,3000],'DL:15.00Mb-UL:3.75Mb':[15000,3750],'DL:18.00Mb-UL:4.50Mb':[18000,4500],'DL:21.00Mb-UL:5.25Mb':[21000,5250]}
dic_cumplimiento=pd.DataFrame.from_dict(dic_cumplimiento, orient='index')
dic_cumplimiento.columns=['Lim_bajada','Lim_subida']
dic_cumplimiento

,Lim_bajada,Lim_subida
DL:12.00Mb-UL:3.00Mb,12000,3000
DL:15.00Mb-UL:3.75Mb,15000,3750
DL:18.00Mb-UL:4.50Mb,18000,4500
DL:21.00Mb-UL:5.25Mb,21000,5250


In [115]:
res_franjas_analitica=Resultados_analitica[Resultados_analitica['CONDICION']=='E'].groupby(['BANDA_x','HORA'])['VELOCIDAD_BAJADA','VELOCIDAD_SUBIDA'].quantile(0.05).reset_index()
res_franjas_analitica=res_franjas_analitica.merge(dic_cumplimiento,left_on='BANDA_x', right_index=True, how='left')
res_franjas_analitica['cump_bajada']=np.where(res_franjas_analitica['VELOCIDAD_BAJADA']>=res_franjas_analitica['Lim_bajada'],1,0)
res_franjas_analitica['cump_subida']=np.where(res_franjas_analitica['VELOCIDAD_SUBIDA']>=res_franjas_analitica['Lim_subida'],1,0)
res_franjas_analitica

,BANDA_x,HORA,VELOCIDAD_BAJADA,VELOCIDAD_SUBIDA,Lim_bajada,Lim_subida,cump_bajada,cump_subida
0,DL:12.00Mb-UL:3.00Mb,6,12601.3250,3843.5710,12000,3000,1,1
1,DL:12.00Mb-UL:3.00Mb,7,13096.9305,4501.4195,12000,3000,1,1
2,DL:12.00Mb-UL:3.00Mb,8,13479.1000,4375.1900,12000,3000,1,1
3,DL:12.00Mb-UL:3.00Mb,9,14170.9740,4696.9460,12000,3000,1,1
4,DL:12.00Mb-UL:3.00Mb,10,13242.3840,4644.2030,12000,3000,1,1
5,DL:12.00Mb-UL:3.00Mb,11,13607.5430,4283.7420,12000,3000,1,1
6,DL:12.00Mb-UL:3.00Mb,12,14110.1810,4334.9610,12000,3000,1,1
7,DL:12.00Mb-UL:3.00Mb,13,13025.7900,4149.4330,12000,3000,1,1
8,DL:12.00Mb-UL:3.00Mb,14,14162.3065,4406.6715,12000,3000,1,1
9,DL:12.00Mb-UL:3.00Mb,15,13970.7610,4177.4270,12000,3000,1,1


In [116]:
res_franjas_analitica.groupby('BANDA_x')['cump_bajada','cump_subida'].sum()

,cump_bajada,cump_subida
BANDA_x,,
DL:12.00Mb-UL:3.00Mb,15,15
DL:15.00Mb-UL:3.75Mb,15,15
DL:18.00Mb-UL:4.50Mb,13,15
DL:21.00Mb-UL:5.25Mb,12,15
